# FinBERT Model Training

This notebook implements the training pipeline for FinBERT model on our financial tweets dataset.

## Pipeline Overview:
1. Load labeled data
2. Preprocess text
3. Configure and initialize FinBERT
4. Train model with evaluation
5. Save model and metrics

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

## 1. Load and Prepare Data

In [ ]:
def load_labeled_data(file_path):
    """Load Gemini-labeled dataset"""
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df)} labeled tweets")
    return df

# Load the labeled data
df = load_labeled_data('../data/labeled_tweets.csv')
df.head()

## 2. Create Dataset and DataLoader

In [ ]:
class FinancialTweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].values, df['label'].values, 
    test_size=0.2, random_state=42
)

# Initialize tokenizer (using FinBERT)
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

# Create datasets
train_dataset = FinancialTweetDataset(train_texts, train_labels, tokenizer)
val_dataset = FinancialTweetDataset(val_texts, val_labels, tokenizer)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

## 3. Model Configuration and Training

In [ ]:
def compute_metrics(eval_pred):
    """Calculate evaluation metrics"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Initialize model
model = AutoModelForSequenceClassification.from_pretrained(
    'ProsusAI/finbert',
    num_labels=3  # Positive, Negative, Neutral
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./finbert_results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./finbert_logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

## 4. Model Evaluation

In [ ]:
def plot_confusion_matrix(cm, classes):
    """Plot confusion matrix using seaborn"""
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes,
                yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Get predictions
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)

# Calculate and display metrics
print("\nClassification Report:")
print(classification_report(val_labels, preds))

# Plot confusion matrix
cm = confusion_matrix(val_labels, preds)
plot_confusion_matrix(cm, classes=['Negative', 'Neutral', 'Positive'])

## 5. Save Model and Results

In [ ]:
# Save the model
output_dir = '../models/finbert'
os.makedirs(output_dir, exist_ok=True)
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# Save evaluation metrics
eval_metrics = compute_metrics((predictions.predictions, val_labels))
metrics_df = pd.DataFrame([eval_metrics])
metrics_df.to_csv(f'{output_dir}/metrics.csv', index=False)

print("\nFinal Evaluation Metrics:")
for metric, value in eval_metrics.items():
    print(f"{metric}: {value:.4f}")